In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, GridSearchCV, cross_val_score
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import xgboost as xgb
from sklearn.metrics.classification import accuracy_score
from sklearn.neighbors.classification import KNeighborsClassifier
from cmath import sqrt
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier, LogisticRegression
import pygeohash as pgh
import matplotlib.pyplot as plt
import nltk
from sklearn.svm import SVR
from copy import deepcopy

import re
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction import text

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from Data import *
from Feature_Engineering import *
from Data_Preprocessing import *

In [3]:
data = Data('Data/train.csv', 'Data/test.csv')

In [4]:
data.data_import()

In [5]:
prepro = Data_Preprocessing()

In [6]:
prepro.X_Y_remove_outliers(data.train)

In [7]:
feat = Feature_Engineering()

In [8]:
feat.extract_dt_time(data.train)

In [9]:
data.train

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,Hour,Year,Month,Minute
0,2013-06-28 17:40:00,SEX OFFENSES FORCIBLE,"FORCIBLE RAPE, BODILY FORCE",Friday,MISSION,NONE,2100 Block of MISSION ST,-122.419331,37.762264,141546,17,2013,6,10
1,2004-02-19 02:46:00,LIQUOR LAWS,CONSUMING ALCOHOL IN PUBLIC VIEW,Thursday,SOUTHERN,"ARREST, BOOKED",1000 Block of MARKET ST,-122.411340,37.781271,794152,2,2004,2,16
2,2007-11-14 00:01:00,FRAUD,"CREDIT CARD, THEFT BY USE OF",Wednesday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421,531205,0,2007,11,-29
3,2007-12-27 18:30:00,ROBBERY,ROBBERY OF A CHAIN STORE WITH A GUN,Thursday,BAYVIEW,DISTRICT ATTORNEY REFUSES TO PROSECUTE,2400 Block of SAN BRUNO AV,-122.404715,37.730161,523137,18,2007,12,0
4,2012-09-09 17:02:00,OTHER OFFENSES,PROBATION VIOLATION,Sunday,SOUTHERN,"ARREST, BOOKED",4TH ST / STEVENSON ST,-122.405239,37.785265,200968,17,2012,9,-28
5,2014-05-14 06:14:00,WARRANTS,ENROUTE TO DEPARTMENT OF CORRECTIONS,Wednesday,MISSION,"ARREST, BOOKED",15TH ST / NATOMA ST,-122.418405,37.766760,75749,6,2014,5,-16
6,2006-08-09 23:45:00,BURGLARY,"BURGLARY, UNLAWFUL ENTRY",Wednesday,CENTRAL,"ARREST, BOOKED",0 Block of MARK LN,-122.404651,37.790442,617521,23,2006,8,15
7,2012-11-16 12:41:00,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,Friday,INGLESIDE,NONE,100 Block of PUTNAM ST,-122.410871,37.737526,186953,12,2012,11,11
8,2007-02-05 23:53:00,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Monday,BAYVIEW,"ARREST, CITED",HALE ST / SAN BRUNO AV,-122.405911,37.732964,584559,23,2007,2,23
9,2006-02-07 07:41:00,DRUG/NARCOTIC,UNDER THE INFLUENCE OF CONTROLLED SUBSTANCES,Tuesday,NORTHERN,"ARREST, BOOKED",ELLIS ST / LARKIN ST,-122.417710,37.784236,653619,7,2006,2,11


In [10]:
data.train = feat.onehot(data.train, ['DayOfWeek', 'Year', 'Hour', 'PdDistrict'])

In [11]:
feat.features

['DayOfWeek_Friday',
 'DayOfWeek_Monday',
 'DayOfWeek_Saturday',
 'DayOfWeek_Sunday',
 'DayOfWeek_Thursday',
 'DayOfWeek_Tuesday',
 'DayOfWeek_Wednesday',
 'Year_2003',
 'Year_2004',
 'Year_2005',
 'Year_2006',
 'Year_2007',
 'Year_2008',
 'Year_2009',
 'Year_2010',
 'Year_2011',
 'Year_2012',
 'Year_2013',
 'Year_2014',
 'Year_2015',
 'Hour_0',
 'Hour_1',
 'Hour_2',
 'Hour_3',
 'Hour_4',
 'Hour_5',
 'Hour_6',
 'Hour_7',
 'Hour_8',
 'Hour_9',
 'Hour_10',
 'Hour_11',
 'Hour_12',
 'Hour_13',
 'Hour_14',
 'Hour_15',
 'Hour_16',
 'Hour_17',
 'Hour_18',
 'Hour_19',
 'Hour_20',
 'Hour_21',
 'Hour_22',
 'Hour_23',
 'PdDistrict_BAYVIEW',
 'PdDistrict_CENTRAL',
 'PdDistrict_INGLESIDE',
 'PdDistrict_MISSION',
 'PdDistrict_NORTHERN',
 'PdDistrict_PARK',
 'PdDistrict_RICHMOND',
 'PdDistrict_SOUTHERN',
 'PdDistrict_TARAVAL',
 'PdDistrict_TENDERLOIN']

In [12]:
feat.add_seasons(data.train)

In [13]:
data.train

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,...,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN,Summer,Winter,Autumn,Spring
0,2013-06-28 17:40:00,SEX OFFENSES FORCIBLE,"FORCIBLE RAPE, BODILY FORCE",Friday,MISSION,NONE,2100 Block of MISSION ST,-122.419331,37.762264,141546,...,0,0,0,0,0,0,1,0,0,0
1,2004-02-19 02:46:00,LIQUOR LAWS,CONSUMING ALCOHOL IN PUBLIC VIEW,Thursday,SOUTHERN,"ARREST, BOOKED",1000 Block of MARKET ST,-122.411340,37.781271,794152,...,0,0,0,1,0,0,0,1,0,0
2,2007-11-14 00:01:00,FRAUD,"CREDIT CARD, THEFT BY USE OF",Wednesday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421,531205,...,0,0,0,1,0,0,0,0,1,0
3,2007-12-27 18:30:00,ROBBERY,ROBBERY OF A CHAIN STORE WITH A GUN,Thursday,BAYVIEW,DISTRICT ATTORNEY REFUSES TO PROSECUTE,2400 Block of SAN BRUNO AV,-122.404715,37.730161,523137,...,0,0,0,0,0,0,0,1,0,0
4,2012-09-09 17:02:00,OTHER OFFENSES,PROBATION VIOLATION,Sunday,SOUTHERN,"ARREST, BOOKED",4TH ST / STEVENSON ST,-122.405239,37.785265,200968,...,0,0,0,1,0,0,0,0,1,0
5,2014-05-14 06:14:00,WARRANTS,ENROUTE TO DEPARTMENT OF CORRECTIONS,Wednesday,MISSION,"ARREST, BOOKED",15TH ST / NATOMA ST,-122.418405,37.766760,75749,...,0,0,0,0,0,0,0,0,0,1
6,2006-08-09 23:45:00,BURGLARY,"BURGLARY, UNLAWFUL ENTRY",Wednesday,CENTRAL,"ARREST, BOOKED",0 Block of MARK LN,-122.404651,37.790442,617521,...,0,0,0,0,0,0,1,0,0,0
7,2012-11-16 12:41:00,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,Friday,INGLESIDE,NONE,100 Block of PUTNAM ST,-122.410871,37.737526,186953,...,0,0,0,0,0,0,0,0,1,0
8,2007-02-05 23:53:00,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Monday,BAYVIEW,"ARREST, CITED",HALE ST / SAN BRUNO AV,-122.405911,37.732964,584559,...,0,0,0,0,0,0,0,1,0,0
9,2006-02-07 07:41:00,DRUG/NARCOTIC,UNDER THE INFLUENCE OF CONTROLLED SUBSTANCES,Tuesday,NORTHERN,"ARREST, BOOKED",ELLIS ST / LARKIN ST,-122.417710,37.784236,653619,...,1,0,0,0,0,0,0,1,0,0


In [14]:
feat.features

['DayOfWeek_Friday',
 'DayOfWeek_Monday',
 'DayOfWeek_Saturday',
 'DayOfWeek_Sunday',
 'DayOfWeek_Thursday',
 'DayOfWeek_Tuesday',
 'DayOfWeek_Wednesday',
 'Year_2003',
 'Year_2004',
 'Year_2005',
 'Year_2006',
 'Year_2007',
 'Year_2008',
 'Year_2009',
 'Year_2010',
 'Year_2011',
 'Year_2012',
 'Year_2013',
 'Year_2014',
 'Year_2015',
 'Hour_0',
 'Hour_1',
 'Hour_2',
 'Hour_3',
 'Hour_4',
 'Hour_5',
 'Hour_6',
 'Hour_7',
 'Hour_8',
 'Hour_9',
 'Hour_10',
 'Hour_11',
 'Hour_12',
 'Hour_13',
 'Hour_14',
 'Hour_15',
 'Hour_16',
 'Hour_17',
 'Hour_18',
 'Hour_19',
 'Hour_20',
 'Hour_21',
 'Hour_22',
 'Hour_23',
 'PdDistrict_BAYVIEW',
 'PdDistrict_CENTRAL',
 'PdDistrict_INGLESIDE',
 'PdDistrict_MISSION',
 'PdDistrict_NORTHERN',
 'PdDistrict_PARK',
 'PdDistrict_RICHMOND',
 'PdDistrict_SOUTHERN',
 'PdDistrict_TARAVAL',
 'PdDistrict_TENDERLOIN',
 'Summer',
 'Winter',
 'Autumn',
 'Spring']

In [15]:
train, test = feat.odds_base_target(data.train, data.test, 'Address', 'Category', 'ac', add_base_odds = True)

(                     Dates               Category  \
 0      2013-06-28 17:40:00  SEX OFFENSES FORCIBLE   
 1      2004-02-19 02:46:00            LIQUOR LAWS   
 2      2007-11-14 00:01:00                  FRAUD   
 3      2007-12-27 18:30:00                ROBBERY   
 4      2012-09-09 17:02:00         OTHER OFFENSES   
 5      2014-05-14 06:14:00               WARRANTS   
 6      2006-08-09 23:45:00               BURGLARY   
 7      2012-11-16 12:41:00         SUSPICIOUS OCC   
 8      2007-02-05 23:53:00         OTHER OFFENSES   
 9      2006-02-07 07:41:00          DRUG/NARCOTIC   
 10     2005-07-14 17:00:00          LARCENY/THEFT   
 11     2006-02-07 03:57:00         OTHER OFFENSES   
 12     2007-01-12 20:00:00              VANDALISM   
 13     2013-01-22 15:30:00                ASSAULT   
 14     2006-09-21 13:59:00          DRUG/NARCOTIC   
 15     2012-09-19 04:50:00          LARCENY/THEFT   
 16     2003-10-04 11:00:00          DRUG/NARCOTIC   
 17     2007-08-26 22:00:00 

In [10]:
feat.bc_wc_oc(data.train)

In [11]:
data.train

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,Hour,Year,Month,Minute,bc_wc_oc
0,2013-06-28 17:40:00,SEX OFFENSES FORCIBLE,"FORCIBLE RAPE, BODILY FORCE",Friday,MISSION,NONE,2100 Block of MISSION ST,-122.419331,37.762264,141546,17,2013,6,10,b
1,2004-02-19 02:46:00,LIQUOR LAWS,CONSUMING ALCOHOL IN PUBLIC VIEW,Thursday,SOUTHERN,"ARREST, BOOKED",1000 Block of MARKET ST,-122.411340,37.781271,794152,2,2004,2,16,b
2,2007-11-14 00:01:00,FRAUD,"CREDIT CARD, THEFT BY USE OF",Wednesday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421,531205,0,2007,11,-29,w
3,2007-12-27 18:30:00,ROBBERY,ROBBERY OF A CHAIN STORE WITH A GUN,Thursday,BAYVIEW,DISTRICT ATTORNEY REFUSES TO PROSECUTE,2400 Block of SAN BRUNO AV,-122.404715,37.730161,523137,18,2007,12,0,b
4,2012-09-09 17:02:00,OTHER OFFENSES,PROBATION VIOLATION,Sunday,SOUTHERN,"ARREST, BOOKED",4TH ST / STEVENSON ST,-122.405239,37.785265,200968,17,2012,9,-28,o
5,2014-05-14 06:14:00,WARRANTS,ENROUTE TO DEPARTMENT OF CORRECTIONS,Wednesday,MISSION,"ARREST, BOOKED",15TH ST / NATOMA ST,-122.418405,37.766760,75749,6,2014,5,-16,o
6,2006-08-09 23:45:00,BURGLARY,"BURGLARY, UNLAWFUL ENTRY",Wednesday,CENTRAL,"ARREST, BOOKED",0 Block of MARK LN,-122.404651,37.790442,617521,23,2006,8,15,b
7,2012-11-16 12:41:00,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,Friday,INGLESIDE,NONE,100 Block of PUTNAM ST,-122.410871,37.737526,186953,12,2012,11,11,w
8,2007-02-05 23:53:00,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Monday,BAYVIEW,"ARREST, CITED",HALE ST / SAN BRUNO AV,-122.405911,37.732964,584559,23,2007,2,23,o
9,2006-02-07 07:41:00,DRUG/NARCOTIC,UNDER THE INFLUENCE OF CONTROLLED SUBSTANCES,Tuesday,NORTHERN,"ARREST, BOOKED",ELLIS ST / LARKIN ST,-122.417710,37.784236,653619,7,2006,2,11,b


In [13]:
data.train.bc_wc_oc.unique()

array(['b', 'w', 'o'], dtype=object)